# Прогнозирование оттока клиентов в сети отелей «Как в гостях»

Перед нами проект, в котором мы должны проанализировать данные и предсказать долю отказов бронирования непослушных клиентов, чтобы минимизировать убытки отельного бизнеса и непосредственно повысить качество его работы. Столбцов много, данных много, задач много, но все решается, главное постепенно переходить к каждой задачи и внимательно расписывать, согласно условиям.

### Откройте файлы с данными

<div class="alert alert-block alert-info">
<b></b> Импортируем основные библиотеки и установим недостоющие
</div>

In [1]:
# импорты сторонних библиотек
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport

# импорты из стандартной библиотеки
import warnings


# импорты модулей текущего проекта
# длина строки до 78 символов
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, precision_score
from matplotlib import pylab as plt
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    StandardScaler)
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    train_test_split)

# настройки
pd.options.mode.chained_assignment = None
warnings.filterwarnings("ignore")

# параметр для создания вседослучайностей
state = 12345

# бюджет на разработку системы прогнозирования
BUDGET = 400_000

# размер депозита (коэффициент от стоимости и затрат)
DEPOSIT = 0.8

# сезонные коэффициенты
SPRING_LEVEL = 1.2
SUMMER_LEVEL = 1.4
STANDART_LEVEL = 1

# стоимость номеров за ночь
PRICE_A = 1_000
PRICE_B = 800
PRICE_C = 600
PRICE_D = 550
PRICE_E = 500
PRICE_F = 450
PRICE_G = 350

# стоимость разового обслуживания
SERVICE_A = 400
SERVICE_B = 350
SERVICE_C = 350
SERVICE_D = 150
SERVICE_E = 150
SERVICE_F = 150
SERVICE_G = 150

ModuleNotFoundError: No module named 'ydata_profiling'

In [ ]:
!pip install -U imbalanced-learn

In [2]:
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss

In [ ]:
# код ревьюера
df_train = pd.read_csv('/datasets/hotel_train.csv') 
df_test = pd.read_csv('/datasets/hotel_test.csv') 

<div class="alert alert-warning">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: если ты работаешь локально, то тогда стоит переписать код таким образом, что бы он запускался и в Jupyter Hub, и локально без внесения дополнительных правок. Для этого можно использовать конструкцию <a href="https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html" target="blank_">try-except</a> для путей файлов или применить библиотеку os:
    
    import os
    
    pth1_train = '/folder_1/data_train.csv'
    pth1_test = '/folder_1/data_test.csv'
    
    pth2_train = '/folder_2/data_train.csv'
    pth2_test = '/folder_2/data_test.csv'
    
    if os.path.exists(pth1_train) and os.path.exists(pth1_test):
        df = pd.read_csv(pth1_train)
        df_test = pd.read_csv(pth1_test)
    elif os.path.exists(pth2_train) and os.path.exists(pth2_test):
        df = pd.read_csv(pth2_train)
        df_test = pd.read_csv(pth2_test)
    else:
        print('Something is wrong')

В этот раз я поправил, чтобы допроверить проект, но в следующий раз будь внимательнее, это считается критичной ошибкой, проект должен целиком запускаться в окружении Практикума
</div>

In [ ]:
df_test_copy = df_test

<div class="alert alert-block alert-info">
<b></b> Посмотрим на тренировочный и тестовый датасет.
</div>

In [ ]:
print('Тренировочная выборка')
print(df_train.info())
print()
print('#######################')
print()
print('Тестовая выборка')
print(df_test.info())

In [ ]:
print('Тренировочная выборка')
print(df_train.describe())
print()
print('#######################')
print()
print('Тестовая выборка')
print(df_test.describe())

<div class="alert alert-block alert-info">
<b></b> Столбцов много, значений много, но мы не боимся и берёмся за первичный анализ.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: данные на месте!</div>

### Предобработка и исследовательский анализ данных

<div class="alert alert-block alert-info">
<b></b> Сделаем важный шаг и посмотрим, что это за зверь ydata_profiling с методом ProfileReport.
</div>

In [ ]:
report = ProfileReport(df_train, title='My Data')
report_test = ProfileReport(df_test, title='My Data Test')

In [ ]:
report

In [ ]:
report_test

<div class="alert alert-block alert-info">
<b>Первичные мысли по отчёту ProfileReport</b> 
    
1) Можно видеть, что в данных отсутствуют пропуски. А это хороший знак.
1.1) Отсутствуют полностью дубликаты. Что говорит о высоком качестве данных (этот признак мы ещё проверим чуть дальше).

2) В большинстве столбцов присутствует большой дизбаланс значений. В дальнейшем это может отразить на обучении модели. 

2.1) Целевой признак также несбалансирован, в дальнейшем будем использовать методы увеличения, уменьшения выборки или    балансировку класса.
  
3) Столец id сильно влияет на корреляцию некоторых столбцов. Его необходимо будем удалить.

3.1) Столбец с указанием года нам не потребуется для обучения модели. Там присутствует только два значения (2015 и 2016 год)
    и должного эффекта на обучение он не окажет, его можем удалить
    
4) Метод ProfileReport позволил оперативно определить список категориальных и количественных переменных. Создадим для них отдельные списки.
  
4) Хоть и метод ProfileReport даёт хорошую визуализацию для первичного анализа данных, некоторые столбцы рассматриваются вручную.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: круто, что используешь профайлер, очень упрощает работу!</div>

<div class="alert alert-block alert-info">
<b></b> Повторно продублируем корреляционную матрицу с некоторыми изменениями для большей наглядности.
</div>

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Корреляция признаков для тренировочной выборки', weight = 'bold')
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(df_train.corr(method='spearman'), annot = True, cmap=cmap)
plt.show()

<div class="alert alert-block alert-info">
<b></b> На представленной корреляционной матрице для тренировочной выборки отчётливо видна хорошая сходимость некоторых признаков между собой,равняется  а именно:
    
- зависимость количества ночей в выходные дни (stays_in_weekend_nights) от общего количества ночей (total_nights), корреляция равняется 0.63;
    
- зависимость количества ночей в будние дни (stays_in_week_nights) от общего количества ночей (total_nights), корреляция равняется 0.87;
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от общего количества ночей (total_nights), корреляция равняется 0.43;
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от количества ночей в будние дни (stays_in_week_nights), корреляция равняется 0.38;
</div>

In [ ]:
plt.figure(figsize=(20, 10))
plt.title('Корреляция признаков для копии тестовой выборки', weight = 'bold')
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(df_test_copy.corr(method='spearman'), annot = True, cmap=cmap)
plt.show()

<div class="alert alert-block alert-info">
<b></b> На представленной корреляционной матрице для копии тестовой выборки отчётливо видна хорошая сходимость некоторых признаков между собой,равняется  а именно:
    
- столбе год прибытия, arrival_date_year никак и ни с чем не коррелирует. Это происходит из-за того, что в тестовой выборки присутствуют данные только за 2017 год. Поэтому сделаем заключением о том, что данный столбец необходимо удалить из тренировочной и копии тестовой выборки, два значения не окажут сильного влияния на модель.
    
- зависимость количества ночей в выходные дни (stays_in_weekend_nights) от общего количества ночей (total_nights), корреляция равняется 0.61;
    
- зависимость количества ночей в будние дни (stays_in_week_nights) от общего количества ночей (total_nights), корреляция равняется 0.88;
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от общего количества ночей (total_nights), корреляция равняется 0.41;
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от количества ночей в будние дни (stays_in_week_nights), корреляция равняется 0.37;
    
Дополнительно можно выделить, что среднии значения корреляционных матриц примерно равны (что касается описанны выше параметров), это может говорить, что выборки относительно друг друга сбалансированы.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: молодец, что изучаешь корреляцию признаков!</div>

In [ ]:
df_train.drop(['arrival_date_year'], axis='columns', inplace=True)
df_test_copy.drop(['arrival_date_year'], axis='columns', inplace=True)

<div class="alert alert-block alert-info">
<b></b> Создадим список для численных и категориальных признаков.
</div>

In [ ]:
num_features = [
          'lead_time', 'stays_in_weekend_nights', 
          'stays_in_week_nights', 'total_of_special_requests', 
          'total_nights'
         ]

obj_features = [
             'arrival_date_month', 'arrival_date_week_number', 
             'arrival_date_day_of_month', 'adults', 'children', 
            'babies', 'meal', 'country', 'distribution_channel', 
             'previous_cancellations', 'previous_bookings_not_canceled',
            'reserved_room_type', 'booking_changes', 
             'days_in_waiting_list', 'customer_type', 
             'required_car_parking_spaces'
            ]

<div class="alert alert-block alert-info">
<b></b> Некоторые столбцы для категориальных переменных могут вызвать сомнения. Объясню, почему добавил такие сюда. К примеру столбцы с число взрослых, детей и малюток имеют небольшой разброс от 0 до 3-ёх четырёх значений. При таком небольшом количестве разброса данных, мы с уверенностью можем отнести их категориальным признакам.
    
Особенно интересен столбец с малюткам, метод ProfileReport позволил сразу увидеть, что все значения в данном столбце сосредоточены на 0. Можно подкорректировать остальные переменные где имеются малютки и привести их к значения 1, что значит присутствует малютка, а если 0, то малютки отсутствуют. Провернём это.
</div>

In [ ]:
df_train.loc[(df_train['babies'] > 1), 'babies'] = 1
df_test_copy.loc[(df_test_copy['babies'] > 1), 'babies'] = 1

In [ ]:
df_train['babies'].unique()

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: верно, так признак будет более показательным, тогда его можно переименовать, например, в `has_babies`, чтобы обозначить его теперь уже бинарную природу</div>

<div class="alert alert-block alert-info">
<b></b> У нас присутствуют также столбцы с числом взрослых и детей. Проверим теорию о том, что дети без родителей могут забронировать отели. Такое в жизни недопустимо, а вот в данных может бывать всякое.
</div>

In [ ]:
df_Fam = df_train[['adults', 'children', 'babies', 'customer_type']]
df_Fam.query('adults==0')

In [ ]:
df_Fam_test = df_test_copy[['adults', 'children', 'babies', 'customer_type']]
df_Fam_test.query('adults==0')

In [ ]:
df_train = df_train.loc[df_train['adults'] != 0]
df_test_copy = df_test.loc[df_test['adults'] != 0]

<div class="alert alert-block alert-info">
<b></b> Теория подтвердилась, такие значения присутствуют, их скоротечно удалим, чтоб такие данные не отразились на обучении дальнейшей модели.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: все так, детям до 14 лет без взрослых сдать номер не могут, тогда все бронирования без взрослых постояльцев похожи на аномалии, которые лучше обработать</div>

<div class="alert alert-block alert-info">
<b></b> Рассмотрим теперь столбец со списком стран, откуда едут клиенты. ProfileReport показал, что список стран невероятно большой, поэтому выделим самые часто встречающиеся, а остальные страны пометим как значение other. Это позволит модели быть чуть точнее и не распыляться на единичные значения. Такое мы уже сделали со столбцами малюток. В other добавим все страны, процент от общего числа которых меньше одной сотой.
</div>

In [ ]:
vc = df_train['country'].value_counts(normalize=True)
df_train.loc[df_train['country'].isin(vc[vc < 0.01].index), 'country'] = 'other'

vc_t = df_test_copy['country'].value_counts(normalize=True)
df_test_copy.loc[df_test_copy['country'].isin(vc[vc < 0.01].index), 'country'] = 'other'

In [ ]:
df_train['country'].unique()

In [ ]:
df_test_copy['country'].unique()

In [ ]:
df_test_copy['country'].value_counts()

In [ ]:
values_c = ['CAF', 'GHA', 'GTM', 'LAO', 'SDN', 'TGO', 'UMI', 'SLE', 'ATF', 'NCL', 'ASM', 'KIR', 'NPL', 'MDG', 'FJI', 'MYT', 'BHS', 'MRT', 'FRO']
df_test_copy = df_test_copy[df_test_copy.country.isin (values_c) == False ]


In [ ]:
df_test_copy['country'].value_counts()

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: с учетом того, что для большинства стран представлено нерепрезентативное количество бронирований, такое решение вполне подходит :)</div>

<div class="alert alert-block alert-info">
<b></b> Удалим столбцы с id клиентов, для обучения модели они никак не понадобятся.
</div>

In [ ]:
df_train.drop(['id'], axis='columns', inplace=True)
df_test_copy.drop(['id'], axis='columns', inplace=True)

<div class="alert alert-block alert-info">
<b></b> После удаления столбца id лучше проверить ещё раз на наличие дубликатов, есть вероятность что они могли появиться (и когда просто пролистывал данные, я их заметил)
</div>

In [ ]:
print(f'Число дубликатов в тренировочной выборке: {df_train.duplicated().sum()}')
print(f'Число дубликатов в тестовой выборке: {df_test_copy.duplicated().sum()}')

In [ ]:
df_train[df_train.duplicated(keep=False)]

In [ ]:
print(f'Число дубликатов в тренировочной выборке: {df_train.duplicated().sum()}')
print(f'Число дубликатов в тренировочной выборке: {df_test_copy.duplicated().sum()}')

<div class="alert alert-block alert-info">
<b></b> Видно, что количество дубликатов неимоверно большое, почти треть выборки это дубликаты. Чтож, на это могут быть различные причины: технические ошибки (в приложении, программе и т.д.), человеческий фактор(люди по ошибки могли дважды забронировать или наоборот групповое бронирование). Для дальнейшего обучения категорически нельзя использовать такую выборку, все дубликаты из тренировочной выборки необходимо немедленно очистить.
</div>

In [ ]:
df_train.drop_duplicates(inplace=True)
df_test_copy.drop_duplicates(inplace=True)

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: согласен, так как дубликаты могут привести к переобучению моделей и смещению метрик(особенно при использовании кросс-валидации), то правильнее их будет удалить</div>

<div class="alert alert-block alert-info">
<b></b> Проверим значения по месяцам прибытия.
</div>

In [ ]:
plt.subplots(1, figsize=(7, 4))
plt.title('Гистограмма распределения бронирования номеров по месяцам для тренировочной выборки')
sns.histplot(data=df_train, x='arrival_date_month', bins=12, hue='is_canceled')
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='light',
    fontsize='medium',
)
plt.show()

plt.subplots(1, figsize=(7, 4))
plt.title('Гистограмма распределения бронирования номеров по месяцам для тестовой выборки')
sns.histplot(data=df_test_copy, x='arrival_date_month', bins=12, hue='is_canceled')
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='light',
    fontsize='large',
)
plt.show()

<div class="alert alert-block alert-info">
<b></b> 
1) Тренировочная выборка    

Анализ показывает, большое количество бронирований приходится на октябрь, а наименьшее на январь и соответсвенно отмена бронирований тоже выпадает на эти месяца. Высокую отмену на январь можно объяснить праздничными днями и неопределённостью людей в плане выезда в определённый отель. Они могут всё поменять по несколько раз буквально в дни вылета и приезда в отель. Можно увидеть такую тенденцию, что к лету число бронирований уменьшается, к осени обратно растёт, к зиме снова резкое падение. Отмена бронирований имеет похожую зависимость, но не такую сильную. Повышенное число бронирований и отмен осенью, можно объяснить концом года, многие отправляют в командировки или по другим рабочим делам. А если это командиовка, бюджет на отель ограничен, многие могли забыть или не учесть этой информации, и затем потом отменяют бронирование.
    
2) Тестовая выборка 

На тестовой выборки немного другая картина. С января расчёт количество броней и доля отмен соответственно растёт. К маю достигается пик, а затем к августу всё  идёт на спад. Количество отмен уменьшается не так явно, как количество броней.

</div>

<div class="alert alert-block alert-info">
<b></b> Проверим значения по неделям прибытия.
</div>

In [ ]:
plt.subplots(1, figsize=(8, 4))
plt.title('Гистограмма распределения бронирования номеров по неделям для тренировочной выборки')
sns.histplot(data=df_train, x='arrival_date_week_number', bins=50, hue='is_canceled')
plt.xticks(
    rotation=0,
    horizontalalignment='right',
    fontweight='medium',
    fontsize='medium',
)
plt.show()

plt.subplots(1, figsize=(8, 4))
plt.title('Гистограмма распределения бронирования номеров по неделям для тестовой выборки')
sns.histplot(data=df_test_copy, x='arrival_date_week_number', bins=35, hue='is_canceled')
plt.xticks(
    rotation=0,
    horizontalalignment='right',
    fontweight='medium',
    fontsize='medium',
)
plt.show()

<div class="alert alert-block alert-info">
<b></b> 
1) Тренировочная выборка    

На тренировочной выборке явно видно два пика, где-то в середине года и ближе к концу. В эти недели количество броней и их отмен достигает своего максимума.
    
2) Тестовая выборка 

На тестовой выборки картина более равномерная, отсутствуют как таковые пики на 'хороших' бронях и отменёных бронях.

</div>

<div class="alert alert-block alert-info">
<b></b> Проверим значения по дням прибытия.
</div>

In [ ]:
plt.subplots(1, figsize=(8, 5))
plt.title('Гистограмма распределения бронирования номеров по дням для тренировочной выборки')
sns.histplot(data=df_train, x='arrival_date_day_of_month', bins=31, hue='is_canceled')
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='medium',
    fontsize='medium',
)
plt.show()

plt.subplots(1, figsize=(8, 5))
plt.title('Гистограмма распределения бронирования номеров по дням для тестовой выборки')
sns.histplot(data=df_test_copy, x='arrival_date_day_of_month', bins=31, hue='is_canceled')
plt.xticks(
    rotation=45,
    horizontalalignment='right',
    fontweight='medium',
    fontsize='medium',
)
plt.show()

<div class="alert alert-block alert-info">
<b></b> 
1) Тренировочная выборка    

Данные распределены нормально, отсутствуют аномалии.
    
2) Тестовая выборка 

Данные распределены нормально, отсутствуют аномалии.

</div>

<div class="alert alert-block alert-info">
<b></b> Рассмотрим ситуацию с клиентами, которые хоть раз до этого бронировали номера.
</div>

In [ ]:
sns.catplot(data=df_train, y='is_repeated_guest', hue='is_canceled', kind='count', height=5, aspect=2).set(title='Тренировочная выборка')
plt.xlabel('Количество повторных бронирований', color='black')
sns.catplot(data=df_test_copy, y='is_repeated_guest', hue='is_canceled', kind='count', height=5, aspect=2).set(title='Тестовая выборка')
plt.xlabel('Количество повторных бронирований', color='black')
plt.show()

<div class="alert alert-block alert-info">
<b></b> Из выборки видно, что клиенты которые уже бронировали отель до этого, тоже могут отменять бронирование. Это видно как на тренировочной и тестовой выборке.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

<div class="alert alert-block alert-info">
<b></b> Ознакомимся со значениям в столбце meal, reserved_room_type. Видно, что в некоторых из них присутствуют лишние пробелы, но ничего, мы поправим это.
</div>

In [ ]:
df_train['meal'].unique()

In [ ]:
df_train['reserved_room_type'].unique()

In [ ]:
df_train['customer_type'].unique() 

In [ ]:
df_train['distribution_channel'].unique() 

In [ ]:
df_train['meal'] = df_train['meal'].str.strip()
df_test_copy['meal'] = df_test_copy['meal'].str.strip()

df_train['reserved_room_type'] = df_train['reserved_room_type'].str.strip()
df_test_copy['reserved_room_type'] = df_test_copy['reserved_room_type'].str.strip()

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: хорошо, что заметил лишние пробелы!</div>

<div class="alert alert-block alert-info">
<b></b> Как и со столбцом с малютками, провернём данную схему для ещё нескольких столбцов. В таких столбцах как: booking_changes, previous_booking_not_canceled, previous_cancellations, day_in_waiting_list, required_car_parking сильно преобладают нулевые значения, и кранйе малая доля отпадает другие значения. Преобразуем данные столбцы так, что 0 соответствует - отсутствием/ненадобностью, а 1 значит есть/имеется/нужно.
</div>

In [ ]:
df_train.loc[(df_train['booking_changes'] > 1), 'booking_changes'] = 1
df_test_copy.loc[(df_test_copy['booking_changes'] > 1), 'booking_changes'] = 1

df_train.loc[(df_train['previous_bookings_not_canceled'] > 1), 
             'previous_bookings_not_canceled'] = 1
df_test_copy.loc[(df_test_copy['previous_bookings_not_canceled'] > 1), 
                 'previous_bookings_not_canceled'] = 1

df_train.loc[(df_train['previous_cancellations'] > 1), 
             'previous_cancellations'] = 1
df_test_copy.loc[(df_test_copy['previous_cancellations'] > 1), 
                 'previous_cancellations'] = 1

df_train.loc[(df_train['days_in_waiting_list'] > 1), 
             'days_in_waiting_list'] = 1
df_test_copy.loc[(df_test_copy['days_in_waiting_list'] > 1), 
                 'days_in_waiting_list'] = 1

df_train.loc[(df_train['required_car_parking_spaces'] > 1), 
             'required_car_parking_spaces'] = 1
df_test_copy.loc[(df_test_copy['required_car_parking_spaces'] > 1), 
                 'required_car_parking_spaces'] = 1

In [ ]:
df_test_copy['required_car_parking_spaces'].unique()

<div class="alert alert-block alert-info">
<b></b> Да, у нас всё получилось, остались только нули и единицы.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: окей, лишний шум в данных будет ни к чему</div>

<div class="alert alert-block alert-info">
<b></b> Построим диаграмму рассеивания для численных признаков.
</div>

In [ ]:
print("                                          ДИАГРАММА РАССЕИВАНИЯ")
pd.plotting.scatter_matrix(df_train[num_features], figsize=(13,13))
plt.show()

<div class="alert alert-block alert-info">
<b></b> В целом диаграмма рассеивания новой информации не принесла. Прослеживается явная корреляция между некоторыми признаками. Данный вывод я делал выше.
</div>

<div class="alert alert-block alert-info">
<b></b> Когда обработали столбцы, стоит проверить нашу выборку для категорий и численных значений, что всё впорядке.
</div>

In [ ]:
df_train[obj_features].sample(5)

In [ ]:
df_train[num_features].sample(5)

<div class="alert alert-block alert-info">
<b></b> Видно, что некоторые значения имеют тип данных float, приведём их всех к int.
</div>

In [ ]:
df_train['lead_time'] = df_train['lead_time'].astype('int64')
df_test_copy['lead_time'] = df_test_copy['lead_time'].astype('int64')

In [ ]:
df_train[['adults', 'children', 'babies']] = df_train[['adults', 'children', 'babies']].astype('int64')
df_test_copy[['adults', 'children', 'babies']] = df_test_copy[['adults', 'children', 'babies']].astype('int64')

<div class="alert alert-block alert-info">
<b>Вывод по второму разделу (предобработка и анализ данных)</b> 

Все ключевые комментарии оставлны в процессе обработки их.

1) Проработы дубликаты, которые встретились в выборках.

2) Определены категориальные и количественные признаки для дальнейшей работы.
    
3) Некоторые столбцы из списка численных значений приведены к категориальным значениям. Это сделано в связи с большим дизбалансом значений в столбце. Теперь в них присутствуют только 0 или 1. 

4) Удалены столбцы с годом заезда и с id пользователей, так как на обучение модели они никак не повлияют и важной информации для обучения не несут. В столбце с годом заезда вообще присутствует одно значение (тестовая выборка) и два значения (тренировочная выборка)
    
5) Подправлены лишние пробелы присутствующие в значениях в столбцах meal и reserved_room_type.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: супер, можно переходить к расчету прибыли</div>

### Вычислите бизнес метрику

<div class="alert alert-block alert-info">
<b></b> В условии задачи от нас требовалось выполнить расчёты на тестовой выборки.
</div>

<div class="alert alert-block alert-info">
<b></b> Создадим функцию которая позволит на определить уровень сезонного коэффициента для выборки.
</div>

In [ ]:
def season_level(months):

    season_level = months

    # летний коэффициент
    season_level.loc[(months == 'June') | (months == 'July') | (months == 'August')] = SUMMER_LEVEL 

    # осенний коэффициент
    season_level.loc[(months == 'September') | (months == 'October') | (months == 'November') |
                     (months == 'March') | (months == 'April') | (months == 'May')] = SPRING_LEVEL

    # стандартный коэффициент
    season_level.loc[(months == 'December') | (months == 'January') | (months == 'February') ] = STANDART_LEVEL

    return season_level

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: согласно заданию, весной и осенью цены повышаются на 20%, а летом — на 40%. Перепроверь, что сезонные коэффициенты применяются корректно</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Подправил. Увидел свою ошибку.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: теперь с сезонными коэффициентами полный порядок :)</div>

<div class="alert alert-block alert-info">
<b></b> Создадим функцию которая позволит на определить cтоимость номера от его класса и соответствующий уровень обслуживания для его.
</div>

In [ ]:
def price_(room_type):

    prices = room_type

    # укажем стоимость номера
    prices.loc[room_type == 'A'] = PRICE_A
    prices.loc[room_type == 'B'] = PRICE_B
    prices.loc[room_type == 'C'] = PRICE_C
    prices.loc[room_type == 'D'] = PRICE_D
    prices.loc[room_type == 'E'] = PRICE_E
    prices.loc[room_type == 'F'] = PRICE_F
    prices.loc[room_type == 'G'] = PRICE_G

    return prices

In [ ]:
def service_(room_type):

    services = room_type

    # укажем стоимость разового обслуживания
    services.loc[room_type == 'A'] = SERVICE_A
    services.loc[room_type == 'B'] = SERVICE_B
    services.loc[room_type == 'C'] = SERVICE_C
    services.loc[room_type == 'D'] = SERVICE_D
    services.loc[room_type == 'E'] = SERVICE_E
    services.loc[room_type == 'F'] = SERVICE_F
    services.loc[room_type == 'G'] = SERVICE_G

    return services

<div class="alert alert-block alert-info">
<b></b> Немного пришлось повозить с созданием копий, а то значения в функции переприсваивались. И я не сразу это понял. Функции service_ и price_ хотел объединить в одну, но где-то функция даёт сбой, и она присваивает только первые значения для стоимости номера для обох случаев. Поэтому пришлось разделить.
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: можно попробовать объединить их, например, так:
    
    def get_prices(row):
        type = row['reserved_room_type']
        if type == 'A':
            row['price_room'] = PRICE_A
            row['cleaning'] = SERVICE_A
        elif type == 'B':
            row['price_room'] = PRICE_B
            row['cleaning'] = SERVICE_B
        ...
        return row
    
    df = df.apply(get_prices, axis=1)
    
Либо, вместо функций, можно использовать словари и метод <a href="https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html" target="blank_">map</a>, например, так:
    
    prices = {
        'A': [PRICE_A, SERVICE_A],
        'B': [PRICE_B, SERVICE_B],
        ...
    }
    
    df['price_room'] = df['reserved_room_type'].map(lambda type: prices[type][0])
    df['cleaning'] = df['reserved_room_type'].map(lambda type: prices[type][1])
    
Но твое решение тоже работает вполне оптимально :)</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Благодарю, что подсказали пару дополнительных вариантов для реализации функции :)
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

In [ ]:
TEST = df_test_copy.copy()

In [ ]:
TEST['season_lvl'] = season_level(df_test_copy.copy()['arrival_date_month'])

In [ ]:
TEST['serv_for_room'] = service_(df_test_copy.copy()['reserved_room_type'])

In [ ]:
TEST['price_for_room'] = price_(df_test_copy.copy()['reserved_room_type'])

In [ ]:
profit = TEST.query('is_canceled == 0')
lesion = TEST.query('is_canceled == 1')

In [ ]:
profit_for_hotel = ((profit['total_nights'] * profit['season_lvl'] * profit['price_for_room']) - 
 (profit['serv_for_room'] * (1 + profit['total_nights']//2))).sum()

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: здесь необходимо доразобраться с количеством уборок, по этому поводу в задании есть две фразы:
- Если клиент снял номер надолго, то убираются каждые два дня.
- Прибыль отеля — это разница между стоимостью номера за все ночи и затраты на обслуживание: как при подготовке номера, так и при проживании постояльца
    
А отсюда мы узнаем три факта:
- в задании не указано, что значит «снял номер надолго», тогда здесь подразумевается любой срок от 2 ночей;
- затраты на обслуживание в случае, когда клиент заселился, нужно вычитать из суммарной стоимости за номер — это у тебя реализовано;
- есть еще одна обязательная уборка перед заселением клиента</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Увидел свою ошибку. Пытался это реализовать через np.ceil (округление до потолка), но видимо не до конца учёл некоторые моменты. Благодарю что подсказали.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: теперь формула для расчета прибыли до внедрения модели составлена верно!</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: тогда для того, чтобы рассчитать количество уборок для любого количества ночей можно использовать формулу `1 + total_nights // 2`. Получается следующее:
    
1 ночь, убираться будут только перед заселением 1 + 1 // 2 = 1
    
2 ночи, убираться будут днем после первой ночи и перед заселением, 1 + 2 // 2 = 2
    
3 ночи, убираться будут днем после первой ночи и перед заселением, 1 + 3 // 2 = 2
    
4 ночи, убираться будут днем после первой ночи, днём после третьей ночи и перед заселением, 1 + 4 // 2 = 3 и т.д.
</div>

In [ ]:
lesion_for_hotel = -(lesion['season_lvl'] * lesion['price_for_room'] + lesion['serv_for_room']).sum()

In [ ]:
print(f'Прибыль отель по тестовой выборке составляет: {int(profit_for_hotel):,} (руб.)')
print(f'Убыток отеля при отмени брони по тестовой выборке составляет: {int(lesion_for_hotel):,} (руб.)')
print(f'Убыток от прибыли составляет: {round(((int(lesion_for_hotel) / int(profit_for_hotel)) * 100), 2)} % ')

<div class="alert alert-block alert-info">
<b>Вывод по третьему разделу (расчёт бизнес метрики)</b> 

Видно, по тестовым данным, убыток от отмены номера составляет почти 10.5 млн.рублей, а в процентах это почти 25% от общей прибыли. Такие существенные убытки необходимо минизировать.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: в остальном с формулой все окей :)</div>

### Формулировка ML-задачи на основе бизнес-задачи

<div class="alert alert-block alert-info">
<b></b> Получим необходимые нам выборки: целевую и остаточную.
</div>

In [ ]:
features_train = df_train.drop('is_canceled', axis=1)
features_test = df_test_copy.drop('is_canceled', axis=1)

target_train = df_train['is_canceled']
target_test = df_test_copy['is_canceled']

<div class="alert alert-block alert-info">
<b></b> Распишем функцию для увеличения выборки. Но для начала посмотрим, на сколько отличается выборка целевого признака, отношение 0 к 1.
</div>

In [ ]:
target_train.value_counts()

In [ ]:
rat = len(target_train.loc[target_train==0])/len(target_train.loc[target_train==1])
rat

<div class="alert alert-block alert-info">
<b></b> Всё таки значение больше к трём, поэтому увеличим выборку в 3 раза. Установим значение repeat=3.
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Up-Downsampling оставлю как пережиток прошлого и учитывать не буду.
</div>

In [ ]:
def upsample(features, target, repeat=3):
    features_zeros=features[target==0]
    features_ones=features[target==1]
    target_zeros=target[target==0]
    target_ones=target[target==1]
    
    features_upsampled=pd.concat([features_zeros]+[features_ones]*repeat)
    target_upsampled=pd.concat([target_zeros]+[target_ones]*repeat)
    features_upsampled, target_upsampled = shuffle(features_upsampled, target_upsampled, random_state=state)
    
    return features_upsampled, target_upsampled

In [ ]:
features_upsampled, target_upsampled = upsample(features_train, target_train)

<div class="alert alert-block alert-info">
<b></b> На гистограмме проверим, что у нас всё получилось и выборки сравнялись.
</div>

In [ ]:
target_upsampled.plot(kind ='hist', bins=2, figsize=(3,3))
plt.title('Соотношение значений выборки после увелечения')
plt.xlabel('Значения выборок')
plt.ylabel('Частот значений')
plt.show()

<div class="alert alert-block alert-info">
<b></b> Распишем функции для уменьшения выборки. Раз в прошлой функции, мы увеличивали выборку в 3 раза, здесь нам надое её уменьшить в 3 раза, чтобы привести к равному диапазону. Установим значение frac=0.33.
</div>

In [ ]:
def downsample(features, target, fraction=0.33):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled, target_downsampled = downsample(features_train, target_train)

In [ ]:
target_downsampled.plot(kind ='hist', bins=2, figsize=(3,3))
plt.title('Соотношение значений выборки после уменьшения')
plt.xlabel('Значения выборок')
plt.ylabel('Частот значений')
plt.show()

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: так как далее модели необходимо сравнивать по метрикам на кросс-валидации, то необходимо учесть, что на upsample/downsample выборках нельзя проводить кросс-валидацию:
- так как upsample дублирует уже имеющиеся объекты, то при кросс-валидации в валидационные фолды могут попасть дубликаты, из-за чего модель почти гарантированно переобучится;
- в случае downsample в фолдах окажутся более сбалансированные выборки, чем исходная, поэтому метрики окажутся завышенными;
    
<a href="https://kiwidamien.github.io/how-to-do-cross-validation-when-upsampling-data.html" target="blank_">Тут</a> можно почитать про это подробнее. Лучшей стратегией будет либо попробовать другие инструменты, как описано в статье, либо использовать другой способ борьбы с дисбалансом, либо реализовать собственную кросс-валидацию</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Попробую методы которые указаны в статье. Благодарю за разъяснение.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

<div class="alert alert-block alert-info">
<b></b> Для категориальнный признаков используем метод OHE,а для численных Scaler
</div>

In [ ]:
num_features = [
          'lead_time', 'stays_in_weekend_nights', 
          'stays_in_week_nights', 'total_of_special_requests', 
          'total_nights'
         ]

obj_features = [
             'arrival_date_month', 'arrival_date_week_number', 
             'arrival_date_day_of_month', 'adults', 'children', 
            'babies', 'meal', 'country', 'distribution_channel', 
             'previous_cancellations', 'previous_bookings_not_canceled',
            'reserved_room_type', 'booking_changes', 
             'days_in_waiting_list', 'customer_type', 
             'required_car_parking_spaces'
            ]

In [ ]:
encoder_ohe = OneHotEncoder(drop='first', sparse = False)
encoder_ohe.fit(features_train[obj_features])

In [ ]:
features_train[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_train[obj_features])
features_train = features_train.drop(obj_features, axis=1)

In [ ]:
features_test[
    encoder_ohe.get_feature_names_out()
] = encoder_ohe.transform(features_test[obj_features])
features_test = features_test.drop(obj_features, axis=1)

In [ ]:
scaler = StandardScaler()
scaler.fit(features_train[num_features]) 
features_train[num_features] = scaler.transform(features_train[num_features])
features_test[num_features] = scaler.transform(features_test[num_features])

<div class="alert alert-block alert-info">
<b></b> Используем новые методы балансировки классов, такие как SMOTE и NearMiss.
</div>

In [ ]:
smt = SMOTE(random_state=state)
features_train_smote, target_train_smote = smt.fit_resample(features_train, target_train)

In [ ]:
nm = NearMiss()
features_train_miss, target_train_miss = nm.fit_resample(features_train, target_train.ravel())

<div class="alert alert-warning">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: такой подход уже больше подходит, но правильнее будет использовать оба метода внутри пайплайнов от imblearn, давай попробуем разобраться чуть подробнее. Суть кросс-валидации в следующем:
- выборка несколько раз делится на обучающий и валидационный фолды;
- после каждого деления модель обучается на обучающем фолде, а оценивается на валидационном;
- итоговой оценкой модели будет служить среднее между получившимися метриками на валидационных фолдах;
    
То есть, если сначала устранить дисбаланс во всей обучающей выборке, а потом проводить кросс-валидацию, то во время кросс-валидации в валидационные фолды могут попасть те объекты, которые синтезировал SMOTE, что может привести к утечке данных. А по аналогии с downsample, NearMiss изменит баланс классов в исходной выборке, из-за чего соотношение классов в валидационных фолдах окажется более сбалансированным, что приведет к смещению метрик.
    
Для того, чтобы избежать этого, нужно убедиться, что SMOTE/NearMiss применяется только к обучающим фолдам внутри кросс-валидации. Как раз для этого используется пайплайн от imblearn, при его использовании алгоритм будет следующий:
- выборка также несколько раз делится на обучающий и валидационный фолд;
- после каждого деления к обучающему фолду применится SMOTE/NearMiss;
- модель обучится на увеличенном/уменьшенном обучающем фолде и оценится на нетронутом валидационном;
- принцип итоговой оценки не поменяется
    
Тогда для корректного устранения дисбаланса при проведении кросс-валидации можно либо использовать пайплайн от imblearn, либо методы, которые не меняют баланс классов в выборке(например, взвешивание классов или изменение порога классификации), либо провести кросс-валидацию вручную(пример был в тренажере в уроке `Машинное обучение в бизнесе — Сбор данных — Кросс-валидация в Python`) и устранить дисбаланс только в обучающем фолде
</div>

<div class="alert alert-block alert-info">
<b></b> Проверим, что у нас прошла обрабокта категориальных и численных признаков.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: здорово, что для OHE используешь именно OneHotEncoder!</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: вообще для «деревянных» моделей лучше использовать порядковое кодирование, так как они заметно быстрее обучаются на меньшем количестве признаков и не теряют в качестве, есть <a href="https://medium.com/data-design/visiting-categorical-features-and-encoding-in-decision-trees-53400fa65931" target="blank_">шикарная статья</a> на эту тему. Но для линейных моделей(в том числе логистической регрессии) порядковое кодирование не подходит, потому что задает приоритет признакам, которого в природе данных нет. Тогда, например, значение 10^2 в признаке X для них будет важнее, чем значение 10^1, а это далеко не всегда верное утверждение. Поэтому для них лучше использовать OHE.
    
Также для «деревянных» моделей масштабирование совсем не обязательно, <a href="https://towardsdatascience.com/how-data-normalization-affects-your-random-forest-algorithm-fbc6753b4ddf" target="blank_">тут</a> можно почитать про это подробнее. Но для линейных моделей наоборот, может заметно улучшить метрики
    
Тогда у тебя получится разная предобработка для разных моделей, это вполне нормально, тут могут помочь пайплайны, они особенно полезны как раз в подобных случаях
    
Подробнее про пайплайны:
* https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html
* https://scikit-learn.org/stable/modules/compose.html
* https://towardsdatascience.com/how-to-use-sklearn-pipelines-for-ridiculously-neat-code-a61ab66ca90d</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Ознакомплюсь с данным материалом уже после выполнения проекта. Очень интересно будет почитать и чуть больше углубиться.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

### Разработка модели ML

<div class="alert alert-block alert-info">
<b></b> Для обучения модели разделим тренировочную выборку на тренирвочную (меньшей доли) и валидационную. 
    
P.S. Возможно это лишние движения, которые только перегружают код, но если так то попрошу поправить меня :)
</div>

In [ ]:
#features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, 
                                                                                #random_state=state
                                                                               #)

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: в этом проекте можно отказаться от выделения валидационной выборки: так как сравнивать модели необходимо по метрикам на кросс-валидации, тогда подбирать гиперпараметры также можно на кросс-валидации, так метрики будут более показательны, а риск переобучения ниже</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Хорошо, отказываюсь от выделения выборки на валидационную и тренировочную. Буду пробовать.
Правильно понимаю, что в данном примере кода:
- model.fit(features_train, target_train)
- predicted_valid = model.predict(features_valid)

В строке features_valid мы вставляем заместо его features_train? Т.е она на себе обучается и на себе же даёт предсказания? Я правильно понял или нет?
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: не совсем, при таком подходе метрики будут не очень показательны, ведь модель уже видела эти объекты на этапе обучения. Поэтому, если отказаться от выделения отдельной валидационной выборке, то модели необходимо сравнивать по метрикам на кросс-валидации. А обучать модель и получать предсказания в таком случае не обязательно, cross_val_score сделает всек самостоятельно, например:

    RANDOM_STATE = 12345
    best_f1 = 0
    best_RFC = None
    
    for est in range(10, 250, 10):
        for depth in range (1, 16, 1):
            model_RFC = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=RANDOM_STATE)
            f1 = cross_val_score(model_RFC, features_train, target_train, cv=5, scoring='f1').mean()
            if f1 > best_f1:
                best_f1 = f1
                best_RFC = model_RFC
    
    print('Лучшая модель:', best_RFC)
    print('F1-мера:', best_f1)
    
В таком случае переобучение здесь не грозит, так как модель несколько раз(в зависимости параметра cv) обучится на обучающих фолдах, а оценка будет проводитcя на валидационном фолде, который не участвовал в обучении

</div>

<div class="alert alert-block alert-info">
<b></b> Для оценки моделей будем использовать метркику F1 меру, так как она полноценно даёт оценить качество описанной модели. Также напишем функцию, которая будет обучать модели и выводить значения метрики.
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: не совсем понятно, что значит «дает полноценно оценить качество», здесь нужно твое пояснение

Вообще выбор метрики всегда зависит от бизнес-задачи, и если ты считаешь, что именно F1-мера лучше других позволит минимизировать убытки от отмен бронирования, то нужно это аргументировать</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b> 
    
Для оценки моделей будем использовать метркику F1 меру, так как она полноценно даёт оценить качество описанной модели. Что я имею ввиду под полноценностью. Помимо F1 меры мы имеем такие метрики как полнота и точность. По отдельности полнота и точность не слишком информативны. Нужно одновременно повышать показатели обеих. 
Полнота и точность оценивают качество прогноза положительного класса с разных позиций. Recall описывает, как хорошо модель разобралась в особенностях этого класса и распознала его. Precision выявляет, не переусердствует ли модель, присваивая положительные метки.А F1 мера это среднее гармоническое полноты и точности. В данном случае для предсказания отказов, я предлагаю выбрать именно метрику F1. 
   
 Также напишем функцию, которая будет обучать модели и выводить значения метрики.
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: окей, принимается, но здесь есть нюанс:
- метрика **precision** показывает, какая доля объектов, которую модель определила как положительные, действительно положительные. То есть, максимизируя метрику precision, мы пытаемся минимизировать количество **ложноположительных** ответов: клиентов, которые не собирались отменять бронирование, а модель решила, что они все-таки отменят.
- метрика **recall** — это доля положительных объектов, которую нашла модель, из всех положительных объектов. Максимизируя recall мы хотим минимизировать количество **ложноотрицательных** ответов модели: клиентов, которые отменят бронирование, но модель решила, что они заселятся.
    
Как мы знаем, отель несет убытки только в случае отмены бронирования(то есть положительных объектов), значит, нам важно найти максимальное количество таких объектов. Тогда для нас важнее всего минимизировать количество ложноотрицательных ответов, чтобы отель мог взять депозит с наибольшего количества «отменяльщиков».
    
Ложноположительные ответы не так критичны, так как депозит просто пойдет в стоимость оплаты. Но тут может возникнуть другая проблема: депозит может отпугнуть клиентов на этапе бронрования, в том числе «надежных». Мы не знаем, какие именно издержки понесет отель в этом случае, тогда стоит попробовать компенсировать это не метрикой, а экспериментом. Для этого можно попробовать проверить такой сценарий, назовем его худшим: пусть половина случайных клиентов из тех, с кого модель предложит взять депозит(вне зависимости от факта отмены), откажутся от услуг. Если сможешь расчитать выручку в таком случае, будет очень наглядно, стоит ли внедрять модель
    
А вообще можно пойти еще дальше: раз основная задача модели — это максимизация прибыли, то почему бы не использовать прибыль в качестве метрики :) Но тогда ее нужно сделать метрикой, для этого можно создать свой scorer с помощью <a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html" target="blank_">make_scorer</a> и передавать его cross_val_score. Использование кастомных метрик выходит за рамки курса, поэтому это не обязательно, но если интересно, то можешь попробовать реализовать самостоятельно, будет здорово :)
</div>

In [ ]:
def train_and_predict(model,  features_train, target_train, features_valid):  
    
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    scores = cross_val_score(model, features_train, target_train, cv=5, scoring='f1')
    final_score = sum(scores) / len(scores) 

    return final_score

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: cross_val_score использует в качестве метрики ту, которая используется для метода score в самой модели. В случае выбранных тобой моделей — это accuracy_score. Так как основная метрика — не accuracy, то, чтобы задать необходимую метрику, можно передать ее в параметр scoring, подробнее про это можно почитать в <a href="https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html" target="blank_">документации</a>, а список поддерживаемых метрик можно посмотреть <a href="https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter" target="blank_">здесь</a>
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

<div class="alert alert-block alert-info">
<b>Логистическая регрессия</b> 
</div>

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', random_state=state)
final_score = train_and_predict(model, features_train, target_train, features_train)
print(f'Лучшее значение F1 после кросс-валидации: {final_score.round(4)}')

In [ ]:
df_results = pd.DataFrame({'Модель': ['Logistic Regression. Нулевая'], 'Score F1 cross-valid': [final_score],
                           'Время обучения (минуты)': [4.27/60], 'Гиперпараметры': [None]})


In [ ]:
df_results

<div class="alert alert-block alert-info">
<b>Случайный лес</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_est = 0
best_scores = 0

count_F1 = []
count_depth = []
count_est = []

for est in range(10, 250, 10):
    for depth in range (1, 16, 1):
        model_RFC = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=state)
        result, final_score = train_and_predict(model_RFC, features_train, target_train, features_train)
        if result > best_res:
            best_res = result
            best_depth = depth
            best_est = est
            best_scores = final_score
        count_F1.append(result)
        count_depth.append(depth)
        count_est.append(est)   

result, final_score 
print(f'Лучшее значение F1-меры при кросс-валидации:{final_score}, оптимальная глубина:{best_depth}, оптимальное разветвление:{best_est}')


In [ ]:
df_results.loc[len(df_results.index)] = ['Случайный лес. Нулевая',
                                         0.44, 63, 'depth=15, est=50']

<div class="alert alert-block alert-info">
<b>МЫСЛИ В СЛУХ</b> 
    
Думаю, что немного перемудрил с выборками, так как прочёл, что при кросс-валидации нет необходимости изначально использовать split для получения выборок. И вопрос, на сколько адекватным будет расчёт F1 меры и средней оценки качества модели (через cross_val_score). Понимаю что разные метрики, но имею ввиду в моём варианте написания. Или можно былоб сначатьа пустить все модели через кросс-валидацию, найти тут, у которой лучше значение и уже к ней расчитать выбранную мной метрику?
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: cross_val_score — это не метрика, а удобный метод для проведения кросс-валидации, внутри которой может использоваться любая метрика. Поэтому, как я писал выше, здесь правильнее всего будет подбирать гиперпараметры и сравнивать модели на кросс-валидации, указав основную метрику в параметре scoring, а от валидационной выборки действительно можно отказаться</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Понял разницу, теперь попробую реализовать правильно это на практике.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

<div class="alert alert-block alert-info">
<b>Дерево решений</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_scores = 0

for depth in range (1, 5, 1):
        model_DTC = DecisionTreeClassifier(max_depth=depth, random_state=state)
        final_score = train_and_predict(model_DTC, features_train, target_train, features_train)
        if final_score > best_scores:
            best_scores = final_score
            best_depth = depth
    
            
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}')

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: для честного эксперимента стоит точнее настроить модели. Постарайся подобрать хотя бы по 2-3 разных гиперпараметра(помимо взвешивания классов) для **каждой** модели, выбранной тобой для исследования</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Чуть больше углубился в пару других параметров по балансировке классов. Благодарю за наставление.
</div>

In [ ]:
df_results.loc[len(df_results.index)] = ['Дерево решений. Нулевая',
                                         0.2042, 5.39/60, 'depth=4']

In [ ]:
df_results

<div class="alert alert-block alert-info">
<b>Промежуточный вывод по первичному обучению моделей</b> 

На удивление, без учёта балансировки классов и других методв увеличения или уменьшения выборки, модель логистической регрессии показала лучшие результаты (значение F1-меры 0.24)  за минимальное время обучения (4 секунды). В сравнении с моделью случайного леса (значение F1-меры 0.231) и временью обучения чуть больше одного часа, это прекрасный показатель.
</div>

#### Взвешивание классов

<div class="alert alert-block alert-info">
<b>Логистическая регрессия</b> 
</div>

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', random_state=state, class_weight='balanced')
final_score = train_and_predict(model, features_train, target_train, features_train)
print(f'Лучшее значение F1-меры после кросс-валидации: {final_score.round(4)}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Logistic Regression. Балансировка классов',
                                         0.331, 4.67/60, None]

<div class="alert alert-block alert-info">
<b>Случайный лес</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_est = 0
best_scores = 0

count_F1 = []
count_depth = []
count_est = []

for est in range(10, 130, 30):
    for depth in range (3, 8, 1):
        model_RFC = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=state, class_weight='balanced')
        final_score = train_and_predict(model_RFC, features_train, target_train, features_train)
        if final_score > best_scores:
            best_depth = depth
            best_est = est
            best_scores = final_score
        count_depth.append(depth)
        count_est.append(est)   

final_score 
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}, оптимальное разветвление:{best_est}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Случайный лес. Балансировка классов',
                                         0.4265, 64, 'depth=4, est=70']

<div class="alert alert-block alert-info">
<b>Дерево решений</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_scores = 0

for depth in range (1, 5, 1):
        model_DTC = DecisionTreeClassifier(max_depth=depth, random_state=state, class_weight='balanced')
        final_score = train_and_predict(model_DTC, features_train, target_train, features_train)
        if final_score > best_scores:
            best_depth = depth
            best_scores = final_score
    
            
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(3)}, оптимальная глубина:{best_depth}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Дерево решений. Балансировка классов',
                                         0.5, 5.95/60, 'depth=4']

<div class="alert alert-block alert-info">
<b>Промежуточный вывод после использования метода балансировки классов</b> 

При сбалансированных классах, модель дерева решений показала лучшие результаты и значение F1 меры сосавило 0.5 при 5 секундах обучения. Лучшие параметры были достигнуты при глубине 5. 
Модель логистической регрессии (0.331) и случайного леса (0.426) показала более худшие результаты .
</div>

#### SMOTE

<div class="alert alert-block alert-info">
<b>Логистическая регрессия</b> 
</div>

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', random_state=state)
final_score = train_and_predict(model, features_train_smote, target_train_smote, features_train_smote)
print(f'Лучшее значение F1-меры после кросс-валидации:{final_score.round(4)}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Logistic Regression. SMOTE',
                                         0.6854, 7/60, None]

<div class="alert alert-block alert-info">
<b>Случайный лес</b> 
</div>

In [ ]:
%%time

best_depth = 0
best_est = 0
best_scores = 0

for est in range(10, 130, 30):
    for depth in range (3, 8, 1):
        model_RFC = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=state)
        final_score = train_and_predict(model_RFC, features_train_smote, target_train_smote, features_train_smote)
        if final_score > best_scores:
            best_depth = depth
            best_est = est
            best_scores = final_score   

best_scores 
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}, оптимальное разветвление:{best_est}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Случайный лес. SMOTE',
                                         0.772, 130, 'depth=7, est=70']

<div class="alert alert-block alert-info">
<b>Дерево решений</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_scores = 0

for depth in range (1, 5, 1):
        model_DTC = DecisionTreeClassifier(max_depth=depth, random_state=state)
        final_score = train_and_predict(model_DTC, features_train_smote, target_train_smote, features_train_smote)
        if final_score > best_scores:
            best_depth = depth
            best_scores = final_score
    
            
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Дерево решений. SMOTE',
                                         0.7453, 8.63/60, 'depth=4']

<div class="alert alert-block alert-info">
<b>Промежуточный вывод после использования метода увеличения целевого признака</b> 

При балансировки методом SMOTE, модель случайного леса показала наилучшие результаты. Значение F1 меры после кросс-валидации составило 0.772 при глубине 7 и числу ветвей 70. Модель дерева решений отличается на пару пунктов, её результаты F1 меры - 0.7453. Модель Логистической регрессии на удивление тоже показала хорошие результаты при данной балансировки и её значение F1 меры составило 0.68.

#### NearMiss

<div class="alert alert-block alert-info">
<b>Логистическая регрессия</b> 
</div>

In [ ]:
%%time
model = LogisticRegression(solver='liblinear', random_state=state)
final_score = train_and_predict(model, features_train_miss, target_train_miss, features_train_miss)
print(f'Лучшее значение F1-меры после кросс-валидации:{final_score.round(4)}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Logistic Regression. NearMiss',
                                         0.464, 1.71/60, None]

<div class="alert alert-block alert-info">
<b>Случайный лес</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_est = 0
best_scores = 0

count_F1 = []
count_depth = []
count_est = []

for est in range(10, 160, 30):
    for depth in range (3, 10, 1):
        model_RFC = RandomForestClassifier(n_estimators=est, max_depth=depth, random_state=state)
        final_score = train_and_predict(model_RFC, features_train_miss, target_train_miss, features_train_miss)
        if final_score > best_scores:
            best_depth = depth
            best_est = est
            best_scores = final_score
        count_depth.append(depth)
        count_est.append(est)   

final_score 
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}, оптимальное разветвление:{best_est}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Случайныйы лес. NearMiss',
                                         0.5342, 44, 'depth=5, est=10']

<div class="alert alert-block alert-info">
<b>Дерево решений</b> 
</div>

In [ ]:
%%time

best_res = 0
best_depth = 0
best_scores = 0

for depth in range (1, 5, 1):
        model_DTC = DecisionTreeClassifier(max_depth=depth, random_state=state)
        final_score = train_and_predict(model_DTC, features_train_miss, target_train_miss, features_train_miss)
        if final_score > best_scores:
            best_depth = depth
            best_scores = final_score
    
            
print(f'Лучшее значение F1-меры после кросс-валидации:{best_scores.round(4)}, оптимальная глубина:{best_depth}')

In [ ]:
df_results.loc[len(df_results.index)] = ['Дерево решений. NearMiss',
                                         0.6924, 2.42/60, 'depth=4']

<div class="alert alert-block alert-info">
<b>Промежуточный вывод после использования метода уменьшения целевого признака</b> 

Модель дерева решений при балансировки методом NearMiss показала наилучшие результаты и значение F1 меры составило 0.69. В то время как модель случайного леса и логистической регрессии имеют более заниженные значения F1 меры (0.53 для случайного леса и 0.46 для логистической регрессии).

In [ ]:
df_results = df_results.round(3)
df_results

<div class="alert alert-block alert-info">
<b>Выбор лучшей модели</b> 

Выборка у нас получилась широкая. Для бизнеса я бы посоветовал остановиться на модели "Случайного леса" с методом SMOTE балансировки классов, т.к. она показала максимальное значение F1-меры=0.777. Оптимальные гиперпараметры: depth=7, est=70. Немного уменьшились максимально значение для гиперпараметров для чуть скорого обучения модели и получие результатов для анализа данного подхода.

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: лайк за табличку с результатами!</div>

In [ ]:
%%time
model = RandomForestClassifier(random_state=state, max_depth=7, n_estimators=70)
model.fit(features_train_smote, target_train_smote)
predicted_test = model.predict(features_test)
print(f'Значение F1-меры на тестовой выборке: {f1_score(target_test, predicted_test).round(3)}')
print(f'Accuracy = {accuracy_score(target_test, predicted_test):.4f}')
print(f'Recall = {recall_score(target_test, predicted_test):.4f}') 
print(f'Precision = {precision_score(target_test, predicted_test):.4f}')

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: на финальном тестировании лучшую модель также необходимо обучить на обучающей выборке, а вот предсказания получать уже для тестовой

Также методы для устранения дисбаланса путем ресемплинга(такие как upsample или downsample) категорически нельзя применять к валидационной/тестовой выборке(по аналогии с кросс-валидацией), иначе метрики будут непоказательны</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Увидел свою ошибку. Подправил. 
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: вот теперь другое дело!</div>

<div class="alert alert-block alert-info">
<b>Результаты тестовой выборки</b> 

Результаты на тестовой выборке оказались не такими замечательными как на тестовой. Значение F1 меры составило 0.565. Точно чуть выше среднего и составляет 0.722. Посмотрим что покажет расчёт прибыли на основе предсказанных данных.

In [ ]:
probabilities_test = model.predict_proba(features_test)
probabilities_one_test = probabilities_test[:, 1]
auc_roc = roc_auc_score(target_test, probabilities_one_test)
print(f'AUC-ROC равняется: {auc_roc.round(3)}')

In [ ]:
fpr, tpr, thresholds = roc_curve(target_test, probabilities_one_test)
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1], linestyle='--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC-кривая')
plt.show()

<div class="alert alert-block alert-info">
<b></b> 
ROC - кривая показывает хорошую форму, доля правильно положительных ответов неспешно растёт. Наша модель отличается от случайной.

#### Шаг 5. Расчёт прибыли

<div class="alert alert-block alert-info">
<b></b> 
Чтобы понять как модель нам принесёт прибыль от её внедрения, нужно вспомнить условия задачи. Прибыль для отеля появляется в тот момент когда клиент отменяет бронирование (сюда входит бронь номера за ночь и затраты на уборку). Внедрение модели выглядит так, мы сравниваем количество отказов в выборки с предсказаниями нашей модели умноженной на депозит который внесёт клиент, если отменет бронь. Таким образом мы посмотрим прибыль которую мы можем получить. Расчёт прибыли будет происходить на тестовой выборке.

In [ ]:
df_predicted_test= pd.DataFrame(predicted_test)

In [ ]:
TEST['predict'] = df_predicted_test

In [ ]:
TEST_p = TEST.copy()

In [ ]:
TEST_p = TEST.copy()

In [ ]:
TEST_p['profit_h'] = 0

In [ ]:
TEST_p.loc[(TEST_p['is_canceled'] == 0) & (TEST_p['predict'] == 0), 'profit_h'] = (TEST_p['price_for_room'] * TEST_p['season_lvl'] * TEST_p['total_nights']) - (TEST_p['serv_for_room'] * (1 + TEST_p['total_nights'] // 2))

In [ ]:
TEST_p.loc[(TEST_p['is_canceled'] == 0) & (TEST_p['predict'] == 1), 'profit_h'] = (TEST_p['price_for_room'] * TEST_p['season_lvl'] * TEST_p['total_nights']) - (TEST_p['serv_for_room'] * (1 + TEST_p['total_nights'] // 2))

In [ ]:
TEST_p.loc[(TEST_p['is_canceled'] == 1) & (TEST_p['predict'] == 0), 'profit_h'] = -((TEST_p['serv_for_room'] + (TEST_p['price_for_room'] * TEST_p['season_lvl'])))

In [ ]:
TEST_p.loc[(TEST_p['is_canceled'] == 1) & (TEST_p['predict'] == 1), 'profit_h'] = (TEST_p['price_for_room'] * TEST_p['season_lvl'] * DEPOSIT) + (TEST_p['serv_for_room'] * DEPOSIT) - (TEST_p['serv_for_room'] + (TEST_p['price_for_room'] * TEST_p['season_lvl']))

In [ ]:
profit_from_ML = TEST_p['profit_h'].sum()

<div class="alert alert-danger">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>

**Нужно доработать** ❌: так как модель будет предсказывать отмену брони не идеально, то для корректного расчета прибыли после внедрения модели, нужно одновременно учитывать факт отмены и предсказания модели. Чтобы не запутаться, можно рассмотреть все возможные варианты:
    
is_canceled = 0, predictions = 0 — постоялец заселился (profit > 0), модель предсказала, что он заселится. В этом случае прибыль отеля никак не меняется.
    
is_canceled = 0, predictions = 1 — постоялец заселился (profit > 0), модель предсказала, что он отменит. В этом случае постоялец внесёт депозит, который пойдёт в счёт оплаты, то есть в этом случае прибыль отеля тоже никак не меняется.
    
is_canceled = 1, predictions = 0 — постоялец отменил бронь (profit < 0), модель предсказала, что он заселится, то есть депозит с такого клиента не брали. Тогда прибыль отеля тоже никак не меняется, отель просто потерял деньги, как и до внедрения модели.
                                                                        
is_canceled = 1, predictions = 1 — постоялец отменил бронь (profit < 0), модель предсказала, что он отменит, с постояльца взяли депозит в 80% от стоимости номера за одни сутки с учетом сезонного коэффициента и затрат на разовую уборку, отель потеряет только 20% от той суммы, которую терял без модели, так как остальное покроет депозит
</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Рассмотрел все варианты, понял, что не учёл. Благодарю за подсказку.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: формула для расчета прибыли после внедрения модели также составлена верно!</div>

<div class="alert alert-block alert-info">
<b></b> 
При анализе данных в первых этапах помню о том, что в тестовой выборке у нас представлены данные только за 8 месяцев. Чтобы получить примерную прибыль за весь год, домножим нашу прибыль на 1.5. Бонусом надо вычесть стоимость разработки системы.

In [ ]:
print(f"Прибыль от внедрения систесы предсказаний отмены номера за тестовый период: {profit_from_ML - BUDGET:,} (рублей)")
print(f"Прибыль от внедрения систесы предсказаний отмены номера за год: {profit_from_ML * 1.5 - BUDGET:,} (рублей)")
print(f"Средняя прибыль в месяц: {round(((profit_from_ML * 1.5 - BUDGET) / 12), 2):,} (рублей)")

<div class="alert alert-block alert-info">
<b>Результаты определния прибыли</b> 

Нехитрым способом мы пришли к тому, что определили прибыль от внедрения системы и она составляет чуть больше 16 млн.рублей за год. Это уже с вычетом выделенного бюджета на разработку системы. Средняя прибыль за год составила 2.1 млн. рублей. На этом значении можно сделать заключение о том, что разработка окупится всего-лишь за месяц, даже меньше. Считаю оправданным вводить данную систему в эксплуатацию.

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Появился вопрос. Выше я расчитывал прибыль по тестововой выборке и там с учётом убытков она составляет 32.5 млн. рублей. После внедрения системы предсказания отказов, прибыль составила 16.5 млн. рублей. Или я немного запутался? или моя модель настолько ушла в минус ?
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: судя по выводу выше, проблема не в формуле а в том, что для не которых строк нет предсказаний модели, из-за чего прибыль оказалась нулевой. Перепроверь, что выборка для расчета прибыли и выборка для предсказаний совпадают по индексам. А добавить предсказания в датафрейм можно еще проще:

    predictions = model.predict(features_test)
    df['predict'] = predictions
    
Вообще модель любого качества не может так уменьшить прибыль, так как внедрение модели меняет выручку только в одном случае: когда клиент отменил бронирование(is_canceled=1) и модель это успешно определила(predictions=1). Получается, суммарная прибыль после внедрения может быть меньше изначальной только на 400000 (если модель не предсказала ни одной отмены бронирования, мы просто потратили деньги на разработку), во всех остальных случаях она будет больше
    
Пожалуйста, поправь этот момент самостоятельно
</div>

In [ ]:
# код ревьюера

TEST_p.sample(5)

### Опишите портрет «ненадёжного» клиента

<div class="alert alert-block alert-info">
<b></b> 

Попробуем оценить наиболее значимые параметры в выборке для ненадёжного клиента. И для наиболее важных параметров представим графики для большего понимания ненадёжного клиента.

In [ ]:
model_skl_iris = RandomForestClassifier()
model_skl_iris.fit(features_test,
                   target_test)

skl_iris_imp = pd.Series(model_skl_iris.feature_importances_,
                        features_test.columns).sort_values()

fig, ax = plt.subplots(figsize=(16,20))
skl_iris_imp.plot.barh(ax=ax)
ax.set_title("Важность признаков")
ax.set_ylabel('Важность')
fig.tight_layout()

<div class="alert alert-block alert-info">
<b></b> 

Да получилось немного коряво, признаю, но наиболее значимые признаки можно увидеть.

P.S. Возникали ошибки когда брал чистый data frame(тренировочный) и что-то вечно не давано. В дальнейшем постараюсь решить с этим проблему, углубиться сильнее в работу данного способа, так как понимаю его наверно важность. Поправьте если нет. Что я имею ввиду под важностью. Чтобы перед обучением модели прогнать через него все параметры и увидеть, какие из них оказывают большее влияние. А остальные менее значимые удалить, чтобы подчистить модель перед обучением и сократить время обучения. 
Правильно мыслю? или ещё есть другие применения.

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: круто, что знаешь про важность признаков! Действительно, feature_importance можно использовать для выбора только нужных признаков, но после подбора гиперпараметров важность может измениться, поэтому алгоритм будет следующим:
- обучить модель и подобрать для нее гиперпараметры на полном датасете(за исключением однозначно не нужных признаков, вроде id)
- изучить важность признаков, отсеять признаки с нулевой важностью
- заново обучить модель только на значимых признаках, это позволит сократить время обучения без потери в качестве.
    
Также изучение важности признаков поможет убедиться, что модель обучилась корректно, сравнив результаты исследовательского анализа с результатами модели.
</div>

<div class="alert alert-warning">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Есть совет** ⚠️: помимо встроенных методов, можно попробовать изучить важность признаков по-другому: например, с помощью библиотеки <a href="https://shap.readthedocs.io/en/latest/index.html">SHAP</a>. Одно из преимушеств этого подхода в том, что он позволяет оценить важность признака независимо от остальных признаков. <a href="https://habr.com/ru/articles/428213/" target="blank_">Здесь</a> можно подробнее почитать про сам принцип алгоритма, а <a href="https://www.kaggle.com/code/wrosinski/shap-feature-importance-with-feature-engineering" target="blank_">тут</a> есть подробный пример использования</div>

<div class="alert alert-block alert-info">
<b>КОММЕНТАРИЙ СТУДЕНТА 1</b>

Благодарю за доп. материал. Ознакомлюсь с ним уже после проекта.
</div>

<div class="alert alert-success">
<h2>Комментарий ревьюера v2<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: 👍</div>

In [ ]:
sns.displot(df_train, x='lead_time', hue='is_canceled', kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Дата бронирования')
plt.show()

In [ ]:
sns.displot(df_train, x='total_nights', hue='is_canceled', bins=10, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Общее число ночей бронирования')
plt.show()

In [ ]:
sns.displot(df_train, x='stays_in_week_nights', hue='is_canceled', bins=6, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('На сколько дней клиент остановится в будние дни')
plt.show()

In [ ]:
sns.displot(df_train, x='total_of_special_requests', hue='is_canceled',bins=5 ,kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Количество дополнительных запросов')
plt.show()

In [ ]:
sns.displot(df_train, x='stays_in_weekend_nights', hue='is_canceled', bins=4, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('На сколько дней клиент остановится в выходные дни')
plt.show()

In [ ]:
sns.displot(df_train, x='distribution_channel', hue='is_canceled', bins=4, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Каналы дистрибуции')

plt.show()

In [ ]:
sns.displot(df_train, x='customer_type', hue='is_canceled', bins=4, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Тип заказчика')
plt.show()

In [ ]:
sns.displot(df_train, x='country', hue='is_canceled', bins=4, kind='hist', multiple='stack', height=7, aspect=7/5)
plt.title('Страна проживания клиента')
plt.show()

<div class="alert alert-block alert-info">
<b>Результаты определния "Ненадёжного клиента"</b> 

- бронирование номера слишком заранее (за 100 дней и более);
- общее число ночей для остановки не превышает 3ёх;
- если в будние дни клиент остаётся не дольше чем на 3 дня;
- если в выходные дни остаётся не дольше чем на 3 дня;
- число дополнительных запросов для отеля (менее 1 или запросы отсутствуют);
- клиент забронировал/узнал отель через дистрибуцию TA/TO;
- тип клиента Transien или Transien-Party;
- страна проживания клиента PRT, GBR, ESP, FRA.

<div class="alert alert-success">
<h2>Комментарий ревьюера<a class="tocSkip"></a></h2>
    
**Отлично!** ✔️: вполне наглядный получился портрет :)</div>

### Напишите общий вывод

<div class="alert alert-block alert-info">
<b>Итоговый вывод по проекту</b> 

Вот и подходит к концу второй сборный проект, подведём основные итоги по нему.
    
1) Анализ данных занял большую часть времени. Данные представлены в хорошем виде. На входе мы имели две выборки, тренировочную и тестовую. В процессе обработки были удалены некоторые столбцы(id, год прибытия) и после появившиеся дубликаты также были устранены. Пропуски в выборках отсутствовали. Устранены также излишние пробелы в столбцах: reserved_room_type и meal.

1.1) Кореляционные матрицы показали хорошую сходимость некоторых признаков, а именно:
    
- зависимость количества ночей в выходные дни (stays_in_weekend_nights) от общего количества ночей (total_nights);
    
- зависимость количества ночей в будние дни (stays_in_week_nights) от общего количества ночей (total_nights);
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от общего количества ночей (total_nights);
    
- зависимость количества дней между датой бронирования и датой прибытия (lead_time) от количества ночей в будние дни (stays_in_week_nights).
    
1.2) Определены числовые и категориальные столбцы в выборках. Часть числовых столбцов превратились категориальные из-за большого дисбаланса нулевых значениях. В таких столбцах остальные значения, которые больше 1, были приравнены к 1. И теперь 0 в данных столбцах значит нет/не было/отсутствует, а 1 значит да/есть.

1.3) Построены гистограммы для связи некоторых переменных и выявления, когда и при каких условиях клиент отменял заказ.
    
1.4) Метод ProfileReport даёт хорошую база для старта анализа данных, упрощает многие моменты для первичного понимания, что происходит с данными и куда следует более детально приглядеться. Хоть он и упрощает старт, практически все столбцы требуется просматривать вручную для больше детальности природы данных.
    
2) Была расчитана бизнес метрика для подсчёта количества убытков и прибыли на тестовой выборке. По тестовым данным, убыток от отмены номера составляет почти 10.5 млн.рублей, а в процентах это почти 30% от общей прибыли. Такие существенные убытки необходимо минизировать. Написана функция для расчёта убытков и прибыли.
    
3)Определена оптимальная модель для расчёта прибыли. Хоть выборка у нас получилась широкая. Для бизнеса я бы посоветовал остановиться на модели "Случайного леса" с увеличением выборки целевого признака, т.к. она показала максимальное значение F1-меры=0.7 и значения оценки кросс-валидации=0.8. Оптимальные гиперпараметры: depth=7, est=70.

3.1) Результаты расчёты данной модели на тестовой выборке оказались лучше ожидаемых. Время обучения составило считанные секунды. Значение F1 меры составило 0.565, что немного ниже чем на тренировочной модели.Значение AUC-ROC составило 0.776. А ROC - кривая показывает хорошую форму, доля правильно положительных ответов уверенно растёт.
    
3.2) Затем на основе этой модели рассчитали ожидаему прибыль от внедрения системы предсказания отказов. Данная прибыль была расчитана по принципу сравнения значений отказов тестовой выборки с предсказанными значениями умноженными на депозит клиент. Расчёты показали, что прибыль от внедрения системы и она составляет чуть больше 25 млн.рублей за год. Это уже с вычетом выделенного бюджета на разработку системы. Средняя прибыль за год составила 2.1 млн. рублей. На этом значении можно сделать заключение о том, что разработка окупится всего-лишь за месяц, даже меньше. Считаю оправданным вводить данную систему в эксплуатацию.

4) Ненадёжный клиент должен соответствовать следующим критериям:
- бронирование номера слишком заранее (за 100 дней и более);
- общее число ночей для остановки не превышает 3ёх;
- если в будние дни клиент остаётся не дольше чем на 3 дня;
- если в выходные дни остаётся не дольше чем на 3 дня;
- число дополнительных запросов для отеля (менее 1 или запросы отсутствуют);
- клиент забронировал/узнал отель через дистрибуцию TA/TO;
- тип клиента Transien или Transien-Party;
- страна проживания клиента PRT, GBR, ESP, FRA.

<div style="padding: 20px 25px; border: 2px #6495ed solid">
    
<h2 style="color: #87187D">Итоговый комментарий ревьюера<a class="tocSkip"></a></h2>

Видно, что ты приложил много усилий, спасибо за проект!

Но тебе обязательно нужно обратить внимание на следующее:
* поправь расчет прибыли до и после внедрения модели;
* не используй upsample/downsample при проведении кросс-валидации;
* укажи основную метрику при вызове cross_val_score;
* поправь этап финального тестирования лучшей модели

Остальные комментарии можно найти в проекте. Готов ответить на любые вопросы :)

<b>Жду доработанный проект на повторное ревью! Удачи!</b>
        
</div>

<div style="padding: 20px 25px; border: 2px #6495ed solid">
    
<h2 style="color: #87187D">Итоговый комментарий ревьюера v2<a class="tocSkip"></a></h2>

То, что нужно, молодец!

Все критичные замечания исправлены, и теперь я могу принять твою работу :) Но не забудь доразобраться с расчетом прибыли после внедрения модели

<b>Успехов в обучении!</b>
        
</div>